In [5]:
import re
import math
import numpy as np
import pandas as pd
from random import *
from tqdm import tqdm
from itertools import chain
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import nltk

# Modeling
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # GPU가 없을 경우 CPU를 사용합니다.
print(f"Using {device}")

Using cuda


In [6]:
PATH = "/kaggle/input/clothing-review/Womens Clothing E-Commerce Reviews.csv"
raw_df = pd.read_csv(PATH, index_col=0)
df = raw_df[['Review Text','Rating']]
df.dropna(inplace=True)
df = df.drop_duplicates()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 22634 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  22634 non-null  object
 1   Rating       22634 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 530.5+ KB


In [7]:
import nltk

nltk.download('stopwords') # NLTK에서 불용어 데이터를 다운로드
stop_words = set(nltk.corpus.stopwords.words('english')) # 영어 불용어 목록을 가져와서 집합으로 저장

# 텍스트 데이터 전처리 함수
def text_preprocessing(text):
    text = text.lower() # 모든 문자를 소문자로 변환
    text = re.sub('<.*?>', '', text) # HTML 태그 제거 (정규식을 사용하여 <와 > 사이의 모든 문자 제거)
    text = re.sub('[^a-zA-Z]', ' ', text) # 알파벳을 제외한 모든 기호 공백으로 대체
    text = [word for word in text.split() if word not in stop_words] # 불용어 제거 (stop_words 리스트에 없는 단어만 유지)
    text = ' '.join(text) # 단어 리스트를 공백으로 구분된 문자열로 다시 결합
    return text # 전처리된 텍스트 반환

df['cleaned_review'] = df['Review Text'].apply(text_preprocessing)

df.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Review Text,Rating,cleaned_review
0,Absolutely wonderful - silky and sexy and comf...,4,absolutely wonderful silky sexy comfortable
1,Love this dress! it's sooo pretty. i happene...,5,love dress sooo pretty happened find store gla...
2,I had such high hopes for this dress and reall...,3,high hopes dress really wanted work initially ...
3,"I love, love, love this jumpsuit. it's fun, fl...",5,love love love jumpsuit fun flirty fabulous ev...
4,This shirt is very flattering to all due to th...,5,shirt flattering due adjustable front tie perf...


In [8]:
df['Rating'] = (df['Rating'] > 3.5).astype(int)

reviews = df['cleaned_review'].tolist()
ratings = df['Rating'].tolist()

def create_vocab_and_tokenize(reviews, min_freq=0, max_length=64):
    # 리뷰를 토큰화합니다.
    def tokenize_text(text):
        return text.split()

    tokenized_reviews = [tokenize_text(review) for review in reviews]

    # 각 토큰의 발생 횟수를 세어봅니다.
    token_counts = Counter(chain(*tokenized_reviews))

    # min_freq 이상 출현한 토큰만으로 어휘를 만듭니다.
    vocab = {token: i for i, (token, count) in enumerate(token_counts.items()) if count >= min_freq}

    # 어휘에 특수 토큰을 추가합니다.
    vocab = {**{'[PAD]': 0, '[UNK]': 1, '[CLS]': 2, '[SEP]': 3}, **vocab}

    # 리뷰를 입력 ID와 attention masks로 변환합니다.
    input_ids = []
    attention_masks = []

    for review in reviews:
        # 리뷰를 토큰화하고 최대 길이에 맞게 잘라냅니다.
        tokens = tokenize_text(review)[:max_length-2]
        # 토큰을 ID로 변환하고, 앞뒤로 [CLS]와 [SEP] 토큰을 추가합니다.
        input_id = [2] + [vocab.get(token, 1) for token in tokens] + [3]
        # attention mask를 생성합니다.
        attention_mask = [1] * len(input_id) + [0] * (max_length - len(input_id))
        # 입력 ID를 max_length에 맞게 [PAD] 토큰으로 채웁니다.
        input_id += [0] * (max_length - len(input_id))
        input_ids.append(input_id)
        attention_masks.append(attention_mask)

    return vocab, torch.tensor(input_ids), torch.tensor(attention_masks)

# `create_vocab_and_tokenize` 함수를 이용하여 주어진 리뷰 데이터를 토큰화하고 어휘를 생성합니다.
vocab, input_ids, attention_masks = create_vocab_and_tokenize(reviews)

# 평점(ratings) 데이터를 PyTorch 텐서로 변환합니다.
ratings = torch.tensor(ratings)

# 아래의 세 줄은 GPU 또는 다른 하드웨어 가속기에 텐서를 옮기기 위한 코드입니다.
# 이를 통해 모델의 학습 및 추론 속도가 향상될 수 있습니다.

input_ids = input_ids.to(device)
attention_masks = attention_masks.to(device)
ratings = ratings.to(device)

In [9]:
# TensorDataset을 이용하여 input_ids, attention_masks, 그리고 ratings을 포함하는 데이터셋을 생성합니다.
dataset = TensorDataset(input_ids, attention_masks, ratings)

# 전체 데이터셋의 90%는 훈련 데이터로, 나머지 10%는 검증 데이터로 사용하기 위한 크기를 계산합니다.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# random_split을 이용하여 데이터셋을 훈련과 검증 데이터셋으로 무작위 분할합니다.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 배치의 크기를 설정합니다.
batch_size = 64

# 훈련 데이터를 위한 DataLoader를 생성합니다. 이때 무작위 샘플링을 사용하여 각 에폭마다 데이터를 무작위로 섞습니다.
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),  # 무작위 샘플링
            batch_size = batch_size
        )

# 검증 데이터를 위한 DataLoader를 생성합니다. 여기서는 순차적 샘플링을 사용하여 데이터 순서를 그대로 유지합니다.
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),  # 순차적 샘플링
            batch_size = batch_size
        )

# 모델 구현
![](https://www.researchgate.net/publication/349546860/figure/fig2/AS:994573320994818@1614136166736/The-Transformer-based-BERT-base-architecture-with-twelve-encoder-blocks.ppm)


In [10]:
# BERT 모델에 사용될 파라미터 설정

maxlen = 512    # 최대 입력 시퀀스 길이 설정. BERT의 표준 입력 길이는 512입니다.
vocab_size = 20000 # len(vocab) # 어휘 크기를 계산합니다. 이는 나중에 임베딩 레이어 생성 시 사용됩니다.
max_pred = 20   # 예측될 최대 토큰 수 설정. 주로 masked language model 학습에서 사용됩니다.

n_layers = 12   # 트랜스포머 모델의 레이어 수
n_heads = 8 # 멀티 헤드 어텐션에서의 헤드 수
d_model = 768   # 임베딩 및 트랜스포머 내부의 은닉 상태 크기
d_ff = 768 * 4  # Feed Forward 네트워크의 차원. 보통 d_model의 4배로 설정됩니다.
d_k = d_v = 64  # 멀티 헤드 어텐션에서 K(=Q)와 V의 차원

n_segments = 2  # 입력 시퀀스의 세그먼트 수 (보통 2: A와 B의 두 가지 세그먼트)
dropout = 0.1   # 모델 내 드롭아웃 비율 설정
lr = 1e-4   # 학습률 설정

epochs = 5  # 학습할 에폭 수

### 활성화 함수

GELU(Gaussian Error Linear Unit)라는 활성화 함수를 구현해봅니다. 

GELU는 BERT와 같은 트랜스포머 모델에서 자주 사용되며, 다른 활성화 함수보다 깊은 신경망에서 잘 동작하는 것으로 알려져 있습니다.

In [11]:
def gelu(x):
    # Hugging Face에서 구현한 gelu 활성화 함수
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

### Embedding layer 클래스 선언
![](https://www.researchgate.net/profile/Akbar-Karimi-4/publication/338934952/figure/fig2/AS:853247933808640@1580441568270/BERT-word-embedding-layer-Devlin-et-al-2018.ppm)

아래는 BERT 모델의 임베딩 레이어를 구현한 것입니다. 임베딩 레이어는 두 가지 주요한 부분으로 구성됩니다:

- 토큰 임베딩(`tok_embedding`): 주어진 입력 토큰(단어나 문자)을 고정된 크기의 벡터로 변환합니다. 이 벡터는 모델이 학습하는 과정에서 업데이트됩니다.

- 위치 임베딩(`pos_embedding`): 트랜스포머 아키텍처는 순차적인 정보를 자동으로 처리하지 않기 때문에, 각 토큰의 위치 정보를 제공하기 위해 위치 임베딩을 사용합니다.

이 두 임베딩은 합쳐져서 각 토큰에 대한 최종 임베딩 벡터를 생성합니다. 그 후, LayerNorm과 Dropout을 통해 임베딩 벡터가 정규화되고, 과적합을 방지하기 위해 일부 노드가 무작위로 0으로 설정됩니다.

In [12]:
class Embedding(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        # 토큰 임베딩: 주어진 단어나 토큰을 d_model 차원의 벡터로 변환
        self.tok_embedding = nn.Embedding(vocab_size, d_model)
        # 위치 임베딩: 각 토큰의 위치를 d_model 차원의 벡터로 변환
        self.pos_embedding = nn.Embedding(maxlen, d_model)

        # 임베딩 후의 벡터를 정규화
        self.norm = nn.LayerNorm(d_model)
        # 드롭아웃을 적용하여 모델의 과적합을 방지
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 입력 x의 시퀀스 길이를 가져옴
        seq_len = x.size(1) # batch_size * seq_len
        # 입력 x의 각 토큰에 대한 위치 정보를 생성
        pos = torch.arange(seq_len, dtype=torch.long, device=x.device) # 0, 1, ..., seq_len-1
        pos = pos.unsqueeze(0).repeat(x.size(0), 1) # batch_size * seq_len

        # 토큰 임베딩과 위치 임베딩을 합침
        embedding = self.tok_embedding(x) + self.pos_embedding(pos) # batch_size * seq_len * d_model

        # 임베딩 벡터를 정규화하고 드롭아웃을 적용
        return self.dropout(self.norm(embedding))

### 3.3 Attention과 QKV

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_15_pUfIgKn.png)

- 쿼리(Query):

퀴리는 현재 주목하고 있는 토큰 또는 정보의 표현입니다. 어텐션 메커니즘에서는 퀴리를 사용하여 다른 모든 토큰 또는 정보와의 관계를 평가합니다.
예를 들어, "나는 ___ 좋아한다"라는 문장에서 빈칸을 채우려 할 때, "나는"이 퀴리가 될 수 있습니다.

- 키(Key):

키는 다른 모든 토큰 또는 정보의 표현입니다. 퀴리와 각 키 사이의 유사도를 계산하여 어떤 토큰 또는 정보에 주목할지 결정합니다.
위의 예에서, 가능한 빈칸의 후보들(예: "사과", "축구", "노래") 각각이 키가 될 수 있습니다.

- 값(Value):

값은 키와 연관된 실제 정보나 페이로드입니다. 어텐션 메커니즘에서는 퀴리와 키 사이의 유사도를 기반으로 각 값에 가중치를 부여하고, 이 가중치를 사용하여 가중 평균된 값을 출력합니다.
위의 예에서, 각 후보에 대한 추가 정보나 설명이 값이 될 수 있습니다.

- 셀프 어텐션(Self attention):

셀프 어텐션은 입력 시퀀스의 각 토큰에 대해 동일한 시퀀스 내의 다른 모든 토큰과의 관계를 계산하는 방식입니다.  

이를 통해 문장 내의 각 단어가 문장의 다른 부분과 어떻게 상호작용하는지를 파악할 수 있습니다.  

셀프 어텐션 연산은 아래와 같이 진행됩니다.  

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_16_neA52rZ.png)

쿼리(Query), 키(Key), 값(Value) 생성:

1. 먼저, 입력 시퀀스의 각 토큰에 대해 쿼리, 키, 값 표현을 생성합니다. 이는 주어진 입력에 대해 선형 변환을 수행함으로써 이루어집니다.

```
self.w_q = nn.Linear(d_model, d_model)
self.w_k = nn.Linear(d_model, d_model)
self.w_v = nn.Linear(d_model, d_model)
```


2. 어텐션 스코어 계산:

각 쿼리와 모든 키 사이의 유사도(주로 내적)를 계산합니다. 이렇게 계산된 스코어는 어떤 토큰들이 현재 주목하고 있는 토큰과 관련이 깊은지를 나타냅니다.

```
energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
```

3. 스코어 정규화 및 소프트맥스 적용:

계산된 스코어를 정규화합니다. 주로 스코어를 특정 값(예: sqrt(d_model))으로 나누어 스케일링합니다. 그 후, 소프트맥스 함수를 적용하여 각 스코어를 확률 값으로 변환합니다.

```
attention = self.dropout(self.softmax(energy))
```

4. 값 가중치 부여 및 합산:

소프트맥스로 얻은 확률 값을 각 값에 곱하여 가중치를 부여합니다. 그런 다음, 모든 가중치가 부여된 값들을 합산하여 셀프 어텐션의 출력을 생성합니다.

```
x = torch.matmul(attention, V)
```

5. 출력 선형 변환:

마지막으로, 어텐션 출력을 추가적인 선형 변환을 통해 최종 결과를 얻습니다.

```
x = self.fc(x)
```

### 3.4 Multihead attention 클래스 선언

트랜스포머 계열의 모델에서 가장 중요한 부분을 꼽으라면, 망설임 없이 멀티 헤드 어텐션 메커니즘이라고 할 수 있습니다.  

멀티 헤드 어텐션은 트랜스포머 아키텍처의 핵심 구성 요소 중 하나로, 셀프 어텐션(Self-Attention) 메커니즘을 여러 번 동시에 수행하는 방식입니다.  

입력 데이터의 다양한 부분에 주목하여 정보를 집계하는 데 사용되며, 특히나 복잡한 패턴과 관계를 학습하는 데 매우 유용합니다

초기화 부분: 쿼리(Q), 키(K), 값(V)에 대한 선형 변환을 정의합니다. 이 변환은 입력 데이터를 여러 '헤드'로 분할하여 각 헤드에서 어텐션을 계산하는 데 사용됩니다.

순전파 부분:

1. 쿼리, 키, 값에 대한 선형 변환을 수행합니다.

2. 멀티 헤드 어텐션을 위해 데이터를 여러 헤드로 분할합니다.

3. 각 헤드에서 어텐션 스코어를 계산하고, 필요한 경우 마스크를 적용합니다.

4. 어텐션 가중치를 계산하고, 이를 사용하여 값 행렬과 곱하여 어텐션 출력을 얻습니다.
5. 모든 헤드의 출력을 연결하고, 추가적인 선형 변환을 수행합니다.

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()

        # 쿼리, 키, 값에 대한 선형 변환
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        # 드롭아웃 적용
        self.dropout = nn.Dropout(dropout)

        # 멀티 헤드 어텐션 후의 선형 변환
        self.fc = nn.Linear(d_model, d_model)

        # 스케일링 팩터
        self.scale = torch.sqrt(torch.FloatTensor([d_model // n_heads])).to(device)
        # 소프트맥스 함수 정의
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]

        # 쿼리, 키, 값에 대한 선형 변환
        Q = self.w_q(query)
        K = self.w_k(key)
        V = self.w_v(value)

        # 멀티 헤드 어텐션을 위한 차원 변환
        Q = Q.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)

        # 어텐션 스코어 계산
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        # 마스크 적용
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        # 소프트맥스 함수를 통해 어텐션 가중치 계산
        attention = self.dropout(self.softmax(energy))

        # 어텐션 가중치와 값 행렬을 곱하여 출력 계산
        x = torch.matmul(attention, V)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(batch_size, -1, d_model)

        # 최종 선형 변환
        x = self.fc(x)

        return x

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()

        # 쿼리, 키, 값에 대한 선형 변환
        # Your code

        # 드롭아웃 적용
        self.dropout = nn.Dropout(dropout)

        # 멀티 헤드 어텐션 후의 선형 변환
        self.fc = nn.Linear(d_model, d_model)

        # 스케일링 팩터
        self.scale = torch.sqrt(torch.FloatTensor([d_model // n_heads])).to(device)
        # 소프트맥스 함수 정의
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]

        # 쿼리, 키, 값에 대한 선형 변환
        # Your code


        # 멀티 헤드 어텐션을 위한 차원 변환
        Q = Q.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)

        # 어텐션 스코어 계산
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        # 마스크 적용
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        # 소프트맥스 함수를 통해 어텐션 가중치 계산
        # Your code


        # 어텐션 가중치와 값 행렬을 곱하여 출력 계산
        x = torch.matmul(attention, V)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(batch_size, -1, d_model)

        # 최종 선형 변환
        x = self.fc(x)

        return x

중간에 멀티 헤드 어텐션을 위해 차원을 변환하는 과정이 있습니다.

`permute()`는 텐서의 차원을 재배열하는 함수로, 여기서 `permute(0, 2, 1, 3)`를 사용하는 이유는 멀티 헤드 어텐션을 계산하기 위해 텐서의 차원을 적절하게 재배열하기 위함입니다.

원래 텐서의 차원은 `[batch_size, seq_len, n_heads, d_model // n_heads]`입니다. 여기서:

- `batch_size`: 배치 크기

- `seq_len`: 시퀀스 길이

- `n_heads`: 어텐션 헤드의 수

- `d_model`: 모델의 차원

permute(0, 2, 1, 3)를 사용하면 차원의 순서가 [batch_size, n_heads, seq_len, d_model // n_heads]로 변경됩니다.  

이렇게 차원을 재배열하면, 각 어텐션 헤드는 독립적으로 시퀀스에 대한 어텐션을 계산할 수 있습니다. 그리고 이렇게 계산된 결과를 다시 원래의 차원 순서로 변경하여 최종 결과를 얻을 수 있습니다.


### 3.5 Positionwise Feedforward Network

PositionwiseFeedforward 네트워크는 트랜스포머 아키텍처의 각 인코더와 디코더 레이어에 포함되어 있습니다. 이 네트워크는 기본적으로 두 개의 선형 변환을 연속적으로 적용하는데, 여기서는 1D Convolution을 사용하여 이 변환을 수행합니다.

![](https://miro.medium.com/max/1906/1*1l5JbeGfEGh2oxjI8koHdQ.png)

초기화 부분: 두 개의 1D Conv 레이어를 정의합니다. 첫 번째 합성곱은 `d_model` 차원의 입력을 `d_ff` 차원으로 확장하고, 두 번째 합성곱은 그 결과를 다시 `d_model` 차원으로 축소합니다.

순전파 부분:

1. 입력 x의 차원을 변경하여 Convolution을 적용하기 적합한 형태로 만듭니다.

2. 첫 번째 합성곱 레이어와 GELU 활성화 함수를 적용한 후, 결과에 드롭아웃을 적용합니다.

3. 두 번째 Convolution 레이어를 적용합니다.

4. 결과의 차원을 원래대로 변경하여 출력합니다.

이 네트워크는 멀티헤드 어텐션의 출력에 비선형 변환을 추가하여 모델의 표현력을 높이는 역할을 합니다.

In [14]:
class PositionwiseFeedforward(nn.Module):
    def __init__(self):
        super().__init__()

        # 1D Convolution을 사용하여 선형 변환을 수행
        self.fc1 = nn.Conv1d(d_model, d_ff, 1)
        self.fc2 = nn.Conv1d(d_ff, d_model, 1)

        # 드롭아웃 정의
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 입력 x의 차원을 변경
        x = x.permute(0, 2, 1)

        # 첫 번째 Convolution과 활성화 함수 적용 후 드롭아웃
        x = self.dropout(gelu(self.fc1(x)))

        # 두 번째 Convolution 적용
        x = self.fc2(x)

        # 차원을 원래대로 변경하여 출력
        x = x.permute(0, 2, 1)

        return x

In [ ]:
class PositionwiseFeedforward(nn.Module):
    def __init__(self):
        super().__init__()

        # 1D Convolution을 사용하여 선형 변환을 수행
        self.fc1 = nn.Conv1d(d_model, d_ff, 1)
        self.fc2 = nn.Conv1d(d_ff, d_model, 1)

        # 드롭아웃 정의
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 입력 x의 차원을 변경
        x = x.permute(0, 2, 1)

        # 첫 번째 Convolution과 활성화 함수 적용 후 드롭아웃
        # Your code

        # 두 번째 Convolution 적용
        # Your code

        # 차원을 원래대로 변경하여 출력
        x = x.permute(0, 2, 1)

        return x

### 3.6 Encoder layer

트랜스포머 아키텍처의 인코더 레이어를 구현한 클래스입니다.  

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_22_teJgoCi.max-800x600.png)

각 인코더 레이어는 앞서 선언한 두 가지 주요 구성 요소로 이루어져 있습니다:
- 멀티 헤드 셀프 어텐션

- Position-wise feedforward network

초기화 부분:

- `MultiHeadAttention`: 멀티헤드 셀프 어텐션을 수행합니다. 이는 입력 시퀀스 내의 각 토큰이 다른 모든 토큰과 어떻게 상호작용하는지를 파악합니다.

- `PositionwiseFeedforward`: 네트워크를 통해 추가적인 비선형 변환을 수행합니다.

- `LayerNorm`: 레이어 정규화는 각 레이어의 출력을 안정화하여 학습을 도와줍니다.

- `Dropout`: 과적합을 방지하기 위한 드롭아웃입니다.

순전파 부분:

- 멀티헤드 셀프 어텐션을 적용한 후, 그 결과와 원래의 입력을 더하고 레이어 정규화를 수행합니다. (잔여 연결 및 레이어 정규화)

- Position-wise Feed forward를 적용한 후, 그 결과와 이전 단계의 출력을 더하고 다시 레이어 정규화를 수행합니다.

이 구조는 BERT 인코더의 각 레이어에서 반복적으로 사용되며, 여러 레이어를 쌓아 복잡한 패턴과 관계를 학습할 수 있게 합니다.


In [15]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder_self_attn = MultiHeadAttention()
        self.encoder_feed_fwd = PositionwiseFeedforward()
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)


    def forward(self, input, input_mask=None):
        # input => [batch_size, seq_len, d_model]

        encoder_outputs = self.layer_norm(input + self.dropout(self.encoder_self_attn(input, input, input, input_mask)))
        encoder_outputs = self.layer_norm(encoder_outputs + self.dropout(self.encoder_feed_fwd(encoder_outputs)))
        return encoder_outputs

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder_self_attn = MultiHeadAttention()
        self.encoder_feed_fwd = PositionwiseFeedforward()
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, input, input_mask=None):
        # input => [batch_size, seq_len, d_model]
        
        # Your code
        """
        1. encoder self attention
        2. dropout
        3. skip connection
        4. layer norm
        """
        
        # Your code
        """
        1. encoder feed forward
        2. dropout
        3. skip connection
        4. layer norm
        """
        
        
        return encoder_outputs

### 3.7 이번 실습에서 사용된 BERT와 원본 모델과의 차이

이번 실습은 분류에 포커스가 맞춰져있기에, 원본 모델과 다소 차이가 있습니다.

1. PositionwiseFeedforward:

원본 모델에서는 Linear layer(Fully-connected layer)을 통해 연산하지만, 이번 실습에서는 1D Convolution layer를 통해 이 부분을 대체합니다.  

1D Convolution layer을 사용하면 시계열의 특성을 자세하게 읽어낼 수 있을 뿐만 아니라, Linear layer에 비해 적은 파라미터로 연산할 수 있습니다.

2. 모델 출력부분 차이

원본 BERT 모델은 주로 두 가지 출력을 제공합니다:

- Sequence Output: 입력 토큰의 각각에 대한 출력을 포함하는 것으로, 이것은 다운스트림 태스크(예: 토큰 수준의 분류)에 사용될 수 있습니다.

- Pooled Output: 첫 번째 토큰([CLS])에 대한 출력으로, 문장 수준의 분류와 같은 다운스트림 태스크에 주로 사용됩니다. 이 출력은 추가적인 dense layer와 tanh 활성화 함수를 통해 얻어집니다.

실습 모델의 출력은 `Pooled_output`에 추가적 변환 없이 바로 사용합니다.:

- 이 모델에서는 BERT의 인코더 부분을 사용하고, 마지막에 분류 작업을 위한 선형 레이어를 추가하였습니다.

- pooled_output = encoder_output[:, 0]: 이 부분은 BERT의 [CLS] 토큰에 해당하는 출력을 가져옵니다. 그러나 원래 BERT에서처럼 추가적인 dense layer와 tanh 활성화 함수를 사용하지 않습니다.

- output = self.classifier(pooled_output): 이 부분은 문장 수준의 분류를 위한 출력을 생성합니다.

### 3.8 BERT 클래스 선언

자 이제 필요한 클래스와 함수를 모두 선언하였습니다.

위 내용들을 바탕으로 BERT 모델을 클래스로 선언하겠습니다.  

초기화 부분:

- `Embedding`: 주어진 단어나 토큰을 벡터 형태로 변환하는 임베딩 레이어입니다.

- `layers`: BERT 모델의 핵심 부분인 인코더 레이어들의 리스트입니다. 각 레이어는 멀티헤드 어텐션과 Position-wise Feedforward 네트워크를 포함합니다.

- `linear, activn, norm`: 추가적인 변환을 위한 레이어와 활성화 함수입니다.

- classifier: 문장의 전체적인 표현을 기반으로 최종 출력을 생성하는 분류기입니다.

순전파 부분:

- 입력 `input_ids`는 `Embedding` 레이어를 통과하여 임베딩 벡터로 변환됩니다.

- 임베딩 출력은 순차적으로 각 `EncoderLayer`를 통과하며, 각 레이어에서는 멀티헤드 어텐션과 Position-wise Feedforward 연산이 수행됩니다.

- 모든 인코더 레이어를 통과한 후, 첫 번째 토큰(보통 `[CLS]` 토큰)의 출력만을 사용하여 문장의 전체적인 표현을 얻습니다.

- 이 표현은 `classifier`를 통과하여 최종 출력을 생성합니다.

In [16]:
# BERT 모델 정의
class BERT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        # 임베딩 레이어: 토큰을 벡터로 변환
        self.embedding = Embedding(vocab_size)
        # 인코더 레이어들의 리스트
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

        # 추가적인 선형 변환
        self.linear = nn.Linear(d_model, d_model)
        # 활성화 함수로 GELU 사용
        self.activn = gelu
        # 레이어 정규화
        self.norm = nn.LayerNorm(d_model)

        # 최종 분류를 위한 선형 레이어
        self.classifier = nn.Linear(d_model, 1)

    def forward(self, input_ids, attention_mask):
        # 임베딩 레이어를 통과
        embedding_output = self.embedding(input_ids)
        # 각 인코더 레이어를 순차적으로 통과
        for layer in self.layers:
            encoder_output = layer(embedding_output)
        # 첫 번째 토큰의 출력만 사용하여 문장의 전체적인 표현을 얻음
        pooled_output = encoder_output[:, 0]
        # 분류를 위해 선형 레이어를 통과
        output = self.classifier(pooled_output)
        return output

In [ ]:
# BERT 모델 정의
class BERT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        # 임베딩 레이어: 토큰을 벡터로 변환
        # Your code
        
        # 인코더 레이어들의 리스트
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

        # 추가적인 선형 변환
        self.linear = nn.Linear(d_model, d_model)
        # 활성화 함수로 GELU 사용
        self.activn = gelu
        # 레이어 정규화
        self.norm = nn.LayerNorm(d_model)

        # 최종 분류를 위한 선형 레이어
        self.classifier = nn.Linear(d_model, 1)

    def forward(self, input_ids, attention_mask):
        # 임베딩 레이어를 통과
        
        # Your code
        
        # 각 인코더 레이어를 순차적으로 통과
        
        # Your code
        
        # 첫 번째 토큰의 출력만 사용하여 문장의 전체적인 표현을 얻음
        
        pooled_output = encoder_output[:, 0]
        
        # 분류를 위해 선형 레이어를 통과
        
        # Your code
        return 

### 3.9 모델 인스턴스화

클래스를 바탕으로 모델 인스턴스를 생성해봅시다.  

- `BERT(vocab_size)`: 주어진 어휘 크기를 사용하여 BERT 모델을 초기화합니다.

`model.to(device)`: 모델을 지정된 디바이스로 이동시킵니다. device는 보통 GPU를 의미하며, GPU에서 모델을 학습하려는 경우 이 코드를 사용하여 데이터와 모델을 GPU로 전송해야합니다.

`nn.BCEWithLogitsLoss()`: 이번 실습은 리뷰에 대한 긍･부정 평가입니다. 이진 분류 문제를 위한 손실 함수입니다. 로짓을 입력으로 받아 이진 교차 엔트로피 손실을 계산합니다.

`optim.Adam(model.parameters(), lr=lr)`: Adam 옵티마이저를 초기화합니다. 옵티마이저는 모델의 파라미터를 업데이트하는 데 사용됩니다.

`print(model)`: 초기화된 BERT 모델의 구조를 출력합니다.



In [17]:
# BERT 모델 초기화
model = BERT(vocab_size)
# 모델을 지정된 디바이스로 이동(GPU)
model = model.to(device)
# 손실 함수로 Binary Cross Entropy with Logits Loss 사용
criterion = nn.BCEWithLogitsLoss()
# 최적화 도구로 Adam 사용
optimizer = optim.Adam(model.parameters(), lr=lr)

# 모델 구조 출력
print(model)

BERT(
  (embedding): Embedding(
    (tok_embedding): Embedding(20000, 768)
    (pos_embedding): Embedding(512, 768)
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (layers): ModuleList(
    (0-11): 12 x EncoderLayer(
      (encoder_self_attn): MultiHeadAttention(
        (w_q): Linear(in_features=768, out_features=768, bias=True)
        (w_k): Linear(in_features=768, out_features=768, bias=True)
        (w_v): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (fc): Linear(in_features=768, out_features=768, bias=True)
        (softmax): Softmax(dim=-1)
      )
      (encoder_feed_fwd): PositionwiseFeedforward(
        (fc1): Conv1d(768, 3072, kernel_size=(1,), stride=(1,))
        (fc2): Conv1d(3072, 768, kernel_size=(1,), stride=(1,))
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [18]:
def train_and_validate(model, train_dataloader, validation_dataloader, epochs, optimizer, criterion):
    # 전체 에포크만큼 학습 및 검증 반복
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 10)

        # 학습 모드 설정
        model.train()
        total_loss = 0
        train_preds, train_labels = [], []

        # 학습 데이터로 학습 진행
        progress_bar = tqdm(train_dataloader, desc='Training', position=0, leave=True)
        for step, batch in enumerate(progress_bar):
            # 배치 데이터를 디바이스에 할당
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # 그래디언트 초기화
            model.zero_grad()

            # 모델에 입력 데이터 전달 및 출력 얻기
            outputs = model(b_input_ids, b_input_mask)

            # 손실 계산
            loss = criterion(outputs.squeeze(), b_labels.float())
            total_loss += loss.item()

            # 그래디언트 계산
            loss.backward()

            # 그래디언트 클리핑
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # 파라미터 업데이트
            optimizer.step()

            # 예측값 및 레이블 저장
            train_preds.extend(torch.sigmoid(outputs).squeeze().detach().cpu().numpy().tolist())
            train_labels.extend(b_labels.squeeze().detach().cpu().numpy().tolist())

            # 진행 바 업데이트
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

        # 평균 학습 손실 출력
        avg_train_loss = total_loss / len(train_dataloader)
        print("\nAverage training loss: {0:.2f}".format(avg_train_loss))

        # 학습 정확도 계산 및 출력
        train_preds = [1 if pred > 0.5 else 0 for pred in train_preds]
        train_acc = accuracy_score(train_labels, train_preds)
        print("Training Accuracy: {0:.2f}".format(train_acc))

        # 검증 모드 설정
        model.eval()
        total_eval_loss = 0
        val_preds, val_labels = [], []

        # 검증 데이터로 검증 진행
        progress_bar = tqdm(validation_dataloader, desc='Validation', position=0, leave=True)
        for batch in progress_bar:
            # 배치 데이터를 디바이스에 할당
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # 그래디언트 계산 비활성화
            with torch.no_grad():
                # 모델에 입력 데이터 전달 및 출력 얻기
                outputs = model(b_input_ids, b_input_mask)

            # 손실 계산
            loss = criterion(outputs.squeeze(), b_labels.float())
            total_eval_loss += loss.item()

            # 예측값 및 레이블 저장
            val_preds.extend(torch.sigmoid(outputs).squeeze().detach().cpu().numpy().tolist())
            val_labels.extend(b_labels.squeeze().detach().cpu().numpy().tolist())

            # 진행 바 업데이트
            progress_bar.set_postfix({'validation_loss': '{:.3f}'.format(loss.item()/len(batch))})

        # 평균 검증 손실 출력
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        print("\nAverage validation loss: {0:.2f}".format(avg_val_loss))

        # 검증 정확도 계산 및 출력
        val_preds = [1 if pred > 0.5 else 0 for pred in val_preds]
        val_acc = accuracy_score(val_labels, val_preds)
        print("Validation Accuracy: {0:.2f}".format(val_acc))

In [19]:
# 학습 시작
train_and_validate(model, train_dataloader, validation_dataloader, epochs, optimizer, criterion)

Epoch 1/5
----------


Training: 100%|██████████| 319/319 [00:51<00:00,  6.20it/s, training_loss=0.081]



Average training loss: 0.42
Training Accuracy: 0.81


Validation: 100%|██████████| 36/36 [00:04<00:00,  8.42it/s, validation_loss=0.168]



Average validation loss: 0.37
Validation Accuracy: 0.83
Epoch 2/5
----------


...


Average validation loss: 0.32
Validation Accuracy: 0.86
Epoch 5/5
----------


Training: 100%|██████████| 319/319 [00:50<00:00,  6.34it/s, training_loss=0.078]



Average training loss: 0.29
Training Accuracy: 0.87


Validation: 100%|██████████| 36/36 [00:04<00:00,  8.50it/s, validation_loss=0.149]


Average validation loss: 0.35
Validation Accuracy: 0.85
